<a href="https://colab.research.google.com/github/kurkur19/Neural_networks_Al_Khmuz_BS_25/blob/main/6lab_NN_Al_Khmuz_BS_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ПРАКТИЧНА РОБОТА 6: ЗГОРТКА. РОЗРОБКА ЗГОРКОВОЇ НЕЙРОННОЇ МЕРЕЖІ


##*Мета роботи:*
засвоїти основи роботи згорткових нейронних мереж
(Convolutional Neural Networks, CNN), їх архітектуру та застосування для
розпізнавання зображень.

Виконала студентка групи БС-25 Аль Хмуз Карина Бассамівна (1 варіант)

##*Теоретична частина:*

***Згорткові нейронні мережі (Convolutional Neural Networks, CNN)*** – це клас
глибоких нейронних мереж, які були спеціально розроблені для обробки даних з
мультиперспективною структурою (багатовимірною, багатоканальною, тощо). У
контексті зображень мультиперспективна структура може бути матрицею
пікселів, де кожен піксель може бути представлений трьома каналами кольору
(червоний, зелений та синій). Однак CNN не обмежені лише обробкою
зображень.

Для чого використовуються згорткові нейронні мережі:

- Обробка зображень: Основна ділянка застосування CNN – це розпізнавання
зображень. Їх використовують для різних завдань, починаючи від базового
розпізнавання об'єктів на зображеннях до дуже специфічних завдань, таких
як діагностика за допомогою медичних зображень.

- Відео аналіз: Ці мережі можуть бути використані для розпізнавання дій у
відео, відстеження об'єктів тощо.
 Детектування об'єктів та семантична сегментація: Це завдання полягає в
визначенні місцезнаходження об'єктів на зображенні та виділенні окремих
об'єктів.

- Задачі обробки природної мови (NLP): Хоча CNN традиційно асоціюються
з зображеннями, вони також можуть бути ефективними в NLP. Наприклад,
вони можуть бути використані для векторизації слів або фраз.

- Розпізнавання звуку: CNN можуть аналізувати спектрограми і
використовуватися для розпізнавання мови, музичних нот або інших
звукових патернів.

Основна перевага CNN перед іншими типами нейронних мереж полягає в їх
здатності автоматично та адаптивно вчити просторові ієрархії з особливостей
зображення, що є критично важливим для обробки зображень з великим розміром
та складністю.

##*Практична частина:*

Крок 1. Завантаження та підготовка даних

In [1]:
!pip install idx2numpy


  Preparing metadata (setup.py) ... done
  Created wheel for idx2numpy: filename=idx2numpy-1.2.3-py3-none-any.whl size=7904 sha256=1f6a4c98ceb9a3da650dec078fc447b1829132058427a4867999bdf47f767a0d
  Stored in directory: /root/.cache/pip/wheels/e0/f4/e7/643fc5f932ec2ff92997f43f007660feb23f948aa8486f1107
Successfully built idx2numpy


In [3]:
from google.colab import files
uploaded = files.upload()


Saving download_data.py to download_data.py


In [5]:
!python download_data.py


Please select a download option:
1) Kuzushiji-MNIST (10 classes, 28x28, 70k examples)
2) Kuzushiji-49 (49 classes, 28x28, 270k examples)
3) Kuzushiji-Kanji (3832 classes, 64x64, 140k examples)
> 1
Please select a download option:
1) MNIST data format (ubyte.gz)
2) NumPy data format (.npz)
> 1
100% 17740/17740 [00:13<00:00, 1288.97KB/s]
100% 29/29 [00:00<00:00, 182.36KB/s]
100% 2970/2970 [00:02<00:00, 1093.65KB/s]
 83% 5/6 [00:00<00:00, 22574.29KB/s]
All dataset files downloaded!


Дані успішно завантажено. Тепер ми можемо використовувати завантажені файли для підготовки даних у Python.

In [6]:
!ls /content


download_data.py  t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
sample_data	  t10k-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


In [7]:
import gzip
import idx2numpy
import numpy as np
import os

# Шлях до файлів
path = "/content"

def load_kmnist():
    with gzip.open(os.path.join(path, 'train-images-idx3-ubyte.gz'), 'r') as f:
        x_train = idx2numpy.convert_from_file(f)
    with gzip.open(os.path.join(path, 'train-labels-idx1-ubyte.gz'), 'r') as f:
        y_train = idx2numpy.convert_from_file(f)
    with gzip.open(os.path.join(path, 't10k-images-idx3-ubyte.gz'), 'r') as f:
        x_test = idx2numpy.convert_from_file(f)
    with gzip.open(os.path.join(path, 't10k-labels-idx1-ubyte.gz'), 'r') as f:
        y_test = idx2numpy.convert_from_file(f)
    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_kmnist()

# Нормалізація
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Переформатування для CNN
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

print(f"Train data shape: {x_train.shape}, Train labels shape: {y_train.shape}")
print(f"Test data shape: {x_test.shape}, Test labels shape: {y_test.shape}")


Train data shape: (60000, 28, 28, 1), Train labels shape: (60000,)
Test data shape: (10000, 28, 28, 1), Test labels shape: (10000,)


Чудово! Дані завантажено і підготовлено правильно. Тепер ми можемо перейти до створення згорткової нейронної мережі (CNN) для класифікації цього датасету.

Крок 2: Побудова згорткової нейронної мережі

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Побудова моделі
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')  # 10 класів у KMNIST
])

# Компіляція моделі
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Опис моделі
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

Крок 3: Навчання моделі

Використовуємо 10 епох для швидкого навчання.

In [ ]:
# Навчання моделі
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=10,
    batch_size=256
)


Epoch 1/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 50s 207ms/step - accuracy: 0.7162 - loss: 0.9329 - val_accuracy: 0.8467 - val_loss: 0.5032
Epoch 2/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 81s 204ms/step - accuracy: 0.9437 - loss: 0.1877 - val_accuracy: 0.8923 - val_loss: 0.3688
Epoch 3/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 80s 196ms/step - accuracy: 0.9649 - loss: 0.1180 - val_accuracy: 0.9072 - val_loss: 0.3230
Epoch 4/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 81s 192ms/step - accuracy: 0.9760 - loss: 0.0806 - val_accuracy: 0.9246 - val_loss: 0.2654
Epoch 5/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 83s 197ms/step - accuracy: 0.9831 - loss: 0.0583 - val_accuracy: 0.9355 - val_loss: 0.2397
Epoch 6/10
 76/235 ━━━━━━━━━━━━━━━━━━━━ 29s 188ms/step - accuracy: 0.9867 - loss: 0.0472